# **DATA PROCESSING**

**INSTALL THE NECESSARY LIBRARIES**

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

**PROCESS THE DATA**

In [ ]:
# Xử lý df_product
df_product = (
    pd.read_csv('Product.csv', encoding='ISO-8859-1', header=None)
    .drop(index=[0, 1])
    .reset_index(drop=True)
)
df_product.columns = df_product.iloc[0]
df_product = (
    df_product[1:]
    .rename(columns={'Code': 'Model code'})
    .iloc[:, 1:10]
    .drop(columns=['Full designation', 'Nick designation', 'Quantity'])  # Xóa cột Quantity
    .replace({'Vi?t Nam': 'Vietnam','GearBox':'Gearbox'})
)

# Xử lý df_import
df_import = (
    pd.read_csv('Import.csv', encoding='ISO-8859-1')
    .rename(columns={
        'Ngày nhập': 'Import date',
        ' Main S/N': 'Main S/N',
        'Tên pallet': 'Pallet'
    })
    .drop(columns=['STT', 'PO', 'Ngày xuất', 'Position', 'Model', 'Second S/N', 'COC', 'Status', 'NOTE'])
    .replace({
        'Vãn ph?ng': 'Văn phòng',
        '? Ngoài': 'Ở ngoài'
    })
    .assign(
        Import_date=lambda x: pd.to_datetime('1899-12-30') + pd.to_timedelta(x['Import date'], unit='D')
    )
    .reset_index(drop=True)
)
df_import.index += 1
df_import.drop(columns=['Import date'], inplace=True)

# Xử lý df_export
df_export = (
    pd.read_csv('Export.csv', encoding='ISO-8859-1')
    .dropna(subset=['STT'])
    .rename(columns={
        'Phiếu xuất kho': 'Export ID',
        'Ngày xuất kho': 'Export date',
        'Khách hàng': 'Customer',
        'Mã sản phẩm': 'Model code',
        'Tên pallet': 'Pallet',
        'S/N': 'Main S/N'
    })
    .drop(columns=['STT', 'Hợp đồng (PO)', 'Tên sản phẩm', 'Ngày hết hạn bảo hành', 'Status', 'Ghi chú'])
)

df_export['Export date'] = pd.to_datetime(df_export['Export date'])
#Check null
#df_product.isnull().sum() #Type 6, Manufactory 61, Origin 42
#df_import.isnull().sum() #Main S/N 810
#df_export.isnull().sum() #Model code 34, Main S/N 5, Pallet 34

# Xử lý giá trị thiếu cho df_product
## Điền giá trị thiếu cho cột Type
type_dict = df_product.groupby('General name')['Type'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).to_dict()

def fill_type(row):
    if pd.isna(row['Type']):
        general_name = row['General name']
        size_first_letter = row['Size'][0]
        same_general_name = df_product[df_product['General name'] == general_name]
        same_size_letter = same_general_name[same_general_name['Size'].str.startswith(size_first_letter)]
        if same_size_letter.empty:
            return type_dict.get(general_name, None)
        elif same_size_letter['Type'].isna().all():
            type_counts = Counter(same_general_name['Type'].dropna())
            for type_name, _ in type_counts.most_common():
                if type_name not in same_size_letter['Type'].values:
                    return type_name
        else:
            return same_size_letter['Type'].mode().iloc[0]
    return row['Type']

df_product['Type'] = df_product.apply(fill_type, axis=1)

## Điền giá trị thiếu cho cột Manufactory
def fill_manufactory(row):
    if pd.isna(row['Manufactory']):
        same_general_type = df_product[(df_product['General name'] == row['General name']) & (df_product['Type'] == row['Type'])]
        same_size_initial = same_general_type[same_general_type['Size'].str.startswith(row['Size'][0])]
        if not same_size_initial['Manufactory'].isna().all():
            return same_size_initial['Manufactory'].dropna().iloc[0]
        manufactory_counts = same_general_type['Manufactory'].value_counts(normalize=True)
        if not manufactory_counts.empty:
            return manufactory_counts.idxmax()
        same_general = df_product[df_product['General name'] == row['General name']]
        manufactory_counts_general = same_general['Manufactory'].value_counts(normalize=True)
        if not manufactory_counts_general.empty:
            return manufactory_counts_general.idxmax()
        return 'Unknown'
    return row['Manufactory']

df_product['Manufactory'] = df_product.apply(fill_manufactory, axis=1)

## Điền giá trị thiếu cho cột Origin
origin_counts = df_product['Origin'].value_counts()
total_count = len(df_product)
missing_count = df_product['Origin'].isna().sum()
fill_counts = (origin_counts / total_count * missing_count).round().astype(int)

while fill_counts.sum() != missing_count:
    diff = missing_count - fill_counts.sum()
    if diff > 0:
        fill_counts[fill_counts.idxmax()] += 1
    else:
        fill_counts[fill_counts.idxmin()] -= 1

fill_values = [origin for origin, count in fill_counts.items() for _ in range(count)]
missing_indices = df_product[df_product['Origin'].isna()].index
df_product.loc[missing_indices, 'Origin'] = fill_values

# Xử lý giá trị thiếu cho df_import
temp_counter = df_import['Model code'].value_counts().to_dict()

def generate_temp_sn(row):
    if pd.isna(row['Main S/N']):
        model_code = row['Model code']
        temp_counter[model_code] += 1
        return f"{model_code}_TEMP_{temp_counter[model_code]}"
    else:
        return row['Main S/N']

df_import['Main S/N'] = df_import.apply(generate_temp_sn, axis=1)

# Xử lý giá trị thiếu cho df_export
df_export.dropna(subset=['Main S/N', 'Model code', 'Pallet'], inplace=True)
df_export.drop_duplicates(inplace=True)

# **PERFORM CALCULATIONS WITH CLEAN DATA**

**CALCULATE THE NUMBER OF PALLETS USED MONTHLY AS A BASIS FOR CALCULATING STORAGE COSTS**

In [ ]:
## Đổi tên cột
df_export.rename(columns={
    'Export date' : 'Month'
},inplace=True)

df_import.rename(columns={
    'Import date' : 'Month'
},inplace=True)

## Đặt lại kiểu định dạng thời gian và lọc dữ liệu cần thiết
df_import['Month'] = pd.to_datetime(df_import['Month'], format='%Y-%m')
df_export['Month'] = pd.to_datetime(df_export['Month'], format='%Y-%m')
df_import['Month'] = df_import['Month'].dt.to_period('M')
df_export['Month'] = df_export['Month'].dt.to_period('M')

df_export = df_export[(df_export['Pallet'] !='Văn phòng')]
df_export = df_export[(df_export['Pallet'] !='Ở Ngoài')]

## Tạo bảng tạm thời chứa Pallet, Model code và Month
temp_df = pd.concat([
    df_import[['Pallet', 'Model code', 'Month','Main S/N']].assign(Status='import'),
    df_export[['Pallet', 'Model code', 'Month','Main S/N']].assign(Status='export')
])
print(temp_df)
## Lấy danh sách các tháng từ cả hai bảng
months = pd.period_range(start=df_import['Month'].min(), end=df_export['Month'].max(), freq='M')
print(months)
## Tính toán số lượng pallet đang được sử dụng đựng hàng nhập cho mỗi tháng
pallet_quantity = [
    temp_df[temp_df['Month'] <= month].query("Status == 'import'")['Pallet'].nunique()
    for month in months
]
## Tạo DataFrame kết quả
df_pallet = pd.DataFrame({
    'Date': months.to_timestamp(),
    'Pallet quantity': pallet_quantity
})

**CALCULATE THE QUANTITY OF GOODS IN AND OUT OF STOCK ON A MONTHLY CYCLE**

In [ ]:
## #Số lượng hàng nhập mỗi tháng
df_im_quantity = df_import.groupby([pd.Grouper(key='Import date', freq='M'),'Model code']).agg(Import_Quantity=('Main S/N', 'count')).reset_index()
print(df_im_quantity)

## Số lượng hàng xuất mỗi tháng
df_ex_quantity = df_export.groupby([pd.Grouper(key='Export date', freq='M'),'Model code']).agg(Export_Quantity=('Main S/N', 'count')).reset_index()
print(df_ex_quantity)

## Đổi tên cột cho 2 dataframe df_im_quantity và df_ex_quantity
df_ex_quantity.rename(columns={
    'Export date' : 'Month'
},inplace=True)

df_im_quantity.rename(columns={
    'Import date' : 'Month'
},inplace=True)

df_inventory = df_im_quantity.set_index('Month').join(df_ex_quantity.set_index('Month'),lsuffix='_im',rsuffix='_ex').reset_index()
import_date = df_inventory['Import date']

## Tạo 1 list có 20 phần tử
import numpy as np
my_list = np.random.random(20)
inventory = list([0] * len(my_list))

#Tạo hàm cho các biến cần thiết
def import_qty_list(df):
  import_qty = []
  for i in range(len(my_list)):
    im_qty = df_inventory['Import_Quantity'][i]
    import_qty.append(im_qty)
  return import_qty

def export_qty_list(df):
  export_qty = []
  for i in range(len(my_list)):
    ex_qty = df_inventory['Export_Quantity'][i]
    export_qty.append(ex_qty)
  return export_qty
#Tìm lượng hàng tồn kho cuối kỳ (tháng): Tồn đầu kỳ + Nhập trong kỳ - Xuất trong kỳ
def calculate_inventory(row = pd.DataFrame()):
    import_qty = import_qty_list(row)
    export_qty = export_qty_list(row)
    inventory = list([0] * len(my_list))  # Khởi tạo danh sách lượng tồn ban đầu
    for i in range(len(my_list)):
        if i == 0:
            inventory[i] = import_qty[i] - export_qty[i]
        else:
            inventory[i] = inventory[i - 1] + import_qty[i] - export_qty[i]
    return inventory

# Áp dụng hàm tính toán lượng tồn cho dataframe df_inventory
df_inventory['Inventory'] = df_inventory.apply(calculate_inventory,axis=1)
final_list = df_inventory['Inventory'][0]
print(final_list)
#Thêm cột 'Inventory' vào dataframe df_inventory
df_inventory['Inventory'] = final_list

**CALCULATE THE AVERAGE STORAGE TIME FOR EACH PRODUCT LINE**

In [ ]:
# Assuming 'merged_import' and 'merged_export' are your import and export DataFrames
# and they include 'General name', 'Type', 'Import date', and 'Export date' columns.

# Convert dates to datetime if not already
merged_import['Import_date'] = pd.to_datetime(merged_import['Import_date'])
merged_export['Export date'] = pd.to_datetime(merged_export['Export date'])

# Merge the import and export data on 'Model code', 'General name', and 'Type'
merged_data = pd.merge(merged_import, merged_export, on=['Model code', 'General name', 'Type'], suffixes=('_imp', '_exp'))

# Calculate the storage duration for each item
merged_data['Storage Duration'] = (merged_data['Export date'] - merged_data['Import_date']).dt.days

# Convert 'Storage Duration' to integer
merged_data['Storage Duration'] = merged_data['Storage Duration'].astype(int)

# Aggregate to find the average storage duration by 'General name' and 'Type'
storage_duration = merged_data.groupby(['General name', 'Type'])['Storage Duration'].mean().reset_index()

# Convert the average storage duration to integer
storage_duration['Storage Duration'] = storage_duration['Storage Duration'].astype(int)

# Rename columns for clarity
storage_duration.rename(columns={'Storage Duration': 'Average Storage Duration (days)'}, inplace=True)